# Dynamic Defense Against Byzantine Poisoning Attacks in Federated Learning

In this notebook we provide the implemetation of the averaging mechanism DDaBA, presented in [this paper](). We also provide a simple example of how to use it in a federated environment. We use [EMNIST Digits](https://www.nist.gov/itl/products-and-services/emnist-dataset) dataset, a popular dataset for the experiments. We federate the data following an I.I.D. distribution over 20 nodes (clients).

### Data reading

In [1]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import shfl

database = shfl.data_base.Emnist()
train_data, train_labels, test_data, test_label = database.load_data()

In [2]:
iid_distribution = shfl.data_distribution.IidDataDistribution(database)
federated_data, test_data, test_labels = iid_distribution.get_federated_data(num_nodes=20, percent=100)

We reshape training and test data in order to fit the required shape. For reshaping the federated data, we use the class FederatedTransformation. 

In [3]:
import numpy as np

class Reshape(shfl.private.FederatedTransformation):
    
    def apply(self, labeled_data):
        labeled_data.data = np.reshape(labeled_data.data, (labeled_data.data.shape[0], labeled_data.data.shape[1], labeled_data.data.shape[2],1))
        
shfl.private.federated_operation.apply_federated_transformation(federated_data, Reshape())

test_data = np.reshape(test_data, (test_data.shape[0], test_data.shape[1], test_data.shape[2],1))

### Deep Learning model

We use a simple deep learning model based on two layers of CNN implemented in Keras.

In [4]:
import tensorflow as tf

def model_builder():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', strides=1, input_shape=(28, 28, 1)))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2, padding='valid'))
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', strides=1))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2, padding='valid'))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.1))
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
    
    return shfl.model.DeepLearningModel(model)

### Federated Aggregator: DDaBA

In this point, we provide the implementation of the DDaBA aggregation mechanism. For that purpose, we overwrite the implementation of the FedAvg aggregator in [ddaba.py](./ddaba.py) and the implementation of the federated government  in [ddaba_fedeerated_gov.py](./ddaba_federated_gov.py).


In [5]:
from ddaba import DDaBA
from ddaba_federated_gov import DDaBAFederatedGovernment

aggregator = DDaBA()
federated_government = DDaBAFederatedGovernment(model_builder, federated_data, aggregator)

### Run the federated algorithm

Finally, we run 10 rounds of learning.

In [6]:
federated_government.run_rounds(10, test_data, test_label)

Accuracy round 0
[0.         0.000875   0.0025     0.0025     0.00325    0.00337499
 0.00449997 0.00512499 0.00787497 0.00874996 0.009      0.01187497
 0.01262498 0.013125   0.014      0.01674998 0.01725    0.01749998
 0.01999998 0.02375001]
0.1
1.0
[0.09090909 0.09090909 0.04545455 0.04545455 0.04545455 0.04545455
 0.04545455 0.04545455 0.04545455 0.04545455 0.04545455 0.04545455
 0.04545455 0.04545455 0.04545455 0.04545455 0.04545455 0.04545455
 0.04545455 0.04545455]

Evaluating global task:
Global model test performance : [0.7090490460395813, 0.899218738079071]



Accuracy round 1
[0.         0.000875   0.00099999 0.00150001 0.00187498 0.00187498
 0.00299996 0.00449997 0.00462496 0.00462496 0.004875   0.00674999
 0.00762498 0.00762498 0.00762498 0.00787497 0.00799996 0.00924999
 0.00999999 0.01875001]
0.05
0.95
[0.1  0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05
 0.05 0.05 0.05 0.05 0.05 0.  ]

Evaluating global task:
Global model test performance : [0.1162132546

[[0.7090490460395813, 0.899218738079071],
 [0.11621325463056564, 0.9715937376022339],
 [0.07790808379650116, 0.980218768119812],
 [0.06707223504781723, 0.984000027179718],
 [0.05585865303874016, 0.9862812757492065],
 [0.05092736706137657, 0.9878125190734863],
 [0.04910089448094368, 0.988531231880188],
 [0.048101622611284256, 0.9885625243186951],
 [0.04725993424654007, 0.9894062280654907],
 [0.04337694123387337, 0.9897187352180481]]